About Dataset
Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

asin - ID of the product, like B000FA64PK
helpful - helpfulness rating of the review - example: 2/3.
overall - rating of the product.
reviewText - text of the review (heading).
reviewTime - time of the review (raw).
reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
reviewerName - name of the reviewer.
summary - summary of the review (description).
unixReviewTime - unix timestamp.
Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis


Best Practises
- Preprocessing And Cleaning
- Train Test Split
- BOW,TFIDF,Word2vec
- Train ML algorithms

## Review Sentiment Analysis 

- WRT the sentence, the review is positive or negative. So, 2 categories : Positive or Negative
- Rating >= 3 : Positive
- Rating < 3 : Negative 

In [89]:
import pandas as pd

In [90]:
data = pd.read_csv('all_kindle_review.csv')

In [91]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [92]:
## Using only 2 data features : reviewText and rating
data = data[['reviewText', 'rating']]

In [93]:
data[['reviewText', 'rating']]

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [94]:
data.isnull().sum() ## no null values

reviewText    0
rating        0
dtype: int64

In [95]:
data['rating'].unique() ## 5 types of ratings (1-5)

array([3, 5, 4, 2, 1], dtype=int64)

In [96]:
data['rating'].value_counts() ## It's a balanced dataset

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Data Preprocessing & Cleaning

In [97]:
## Positive : 1 & Negative : 0
data['rating'] = data['rating'].apply(lambda x : 0 if x < 3 else 1 )

In [98]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [99]:
data['rating'].value_counts() ## balanced dataset

rating
1    8000
0    4000
Name: count, dtype: int64

In [100]:
## Lowering the case
data['reviewText'] = data['reviewText'].str.lower()

In [101]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [102]:
## Cleaning Data : Removing special characters
import re
data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))

In [103]:
## Cleaning Data : Removing stopwords
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Palak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [104]:
## Cleaning data : Removing stopwords
data['reviewText']=data['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stop_words]))

In [105]:
## Cleaning data : Removing url
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

In [106]:
## Cleaning data : Removing html tags
from bs4 import BeautifulSoup
data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

C:\Users\Palak\AppData\Local\Temp\ipykernel_37312\2568648919.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [ ]:
## Cleaning data : Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

In [108]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [109]:
## Lemmatization

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [110]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [111]:
data['reviewText'] = data['reviewText'].apply(lambda x : lemmatize_words(x))

In [112]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [113]:
## Train - Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['reviewText'], data['rating'], test_size=0.20, random_state=42)

In [114]:
print(X_train)

9182     looking forward book came double space every p...
11091    already owned book spouse forgot already part ...
6428     cool forgot request rate came make mine unreli...
288      short short story basically scene party one ni...
2626     secret service agent secrests even longer serv...
                               ...                        
11964    downloaded book reading review usually reading...
5191     far one hottest book ive ever gotten hand ondo...
5390     even though book free reservation based majori...
860      little mushy 34must take care woman folk34 cha...
7270     book good good set charaterswith background le...
Name: reviewText, Length: 9600, dtype: object


In [ ]:
## Creating Vectors : Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf,y_train)

In [119]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [120]:
y_pred_bow = nb_model_bow.predict(X_test_bow)

In [121]:
y_pred_tfidf = nb_model_bow.predict(X_test_tfidf)

In [ ]:
confusion_matrix(y_test, y_pred_bow)

array([[499, 304],
       [717, 880]], dtype=int64)

In [123]:
print("BOW accuracy : ",accuracy_score(y_test, y_pred_bow))

BOW accuracy :  0.5745833333333333


In [124]:
confusion_matrix(y_test, y_pred_tfidf)

array([[490, 313],
       [710, 887]], dtype=int64)

In [125]:
print("TFIDF accuracy: ", accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.57375
